In [1]:
%pip install openai pandas

You should consider upgrading via the '/Users/alexc/Desktop/UofG/Final Project/FinalProject/runModel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

### Load the data from the JSONs

In [6]:
import os
import json
import pandas as pd

directory_path = os.getcwd() + "/../refactoring-data/refactoring-toy-example/"
data_frames = []

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    if os.path.isfile(file_path) and file_path[-4:] == "json":
        with open(file_path) as f:
            data = json.load(f)

        df = pd.json_normalize(data)  # Convert JSON to DataFrame
        data_frames.append(df)

In [7]:
print(data_frames[2]["description"][0])


Pull Up Method	public getName() : int from class org.reptile.Turtle to public getName() : int from class org.reptile.Reptile


### Construct Prompt
1. Run the prompt without specifying the lines
2. Run the prompt with specifying the lines where the refactoring has been performed (if they are available)

In [40]:
def join_file(file_lines):
  return '\n'.join(file_lines)

def construct_simple_prompt(file, changes_start=None, changes_end=None):
  if changes_end:
    return "Refactor the following class between the lines " + str(changes_start) + " and " + str(changes_end) + ":\n" + file
  else:
    return "Refactor the following class:\n" + file


#print( construct_simple_prompt(join_file(data_frames[2]["beforeRefactoring.file"])) )
#print( join_file(data.loc[3,"afterRefactoring.file"]) )

### Prompt ChatGPT Turbo 16k

In [73]:
import time

def askChatGPT( prompt ):
  client = OpenAI()

  response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
      {"role": "system", "content": "You are a code quality analyst. Pay close attention to the readability, code smells, and cognitive complexity. Your goal is to optimize the code without changing the functionality. No explanations are needed."},
      {"role": "user", "content": prompt }
    ]
  )
  print(response.choices[0].message.content)
  time.sleep(20)
  return response.choices[0].message.content


In [74]:

def prompt_refactorings(refactorings):
  i = 0
  for ref in refactorings:
    if "beforeRefactoring.file" in ref.columns:
      prompt1 = construct_simple_prompt(join_file(ref["beforeRefactoring.file"][0]))
      LLM_answer = askChatGPT(prompt1)
      ref['LLM_simple_output'] = LLM_answer

      if "beforeRefactoring.startLine" in ref.columns and "beforeRefactoring.endLine" in ref.columns:
        prompt2 = construct_simple_prompt(join_file(ref["beforeRefactoring.file"][0]), ref["beforeRefactoring.startLine"][0], ref["beforeRefactoring.endLine"][0])
        LLM_answer = askChatGPT(prompt2)
        ref['LLM_simple_output_with_columns'] = LLM_answer

      ref.to_json(f"{directory_path}with-answers/{i}_LLM_results.json", orient='records', lines=True)
      print(f"DataFrame {i} successfully saved!")
      i += 1

prompt_refactorings(data_frames)

```java
package org.animals;

public class Labrador extends Dog {
	
}
```
```java
import org.animals.Dog;

public class Labrador extends Dog {
	
}
```
DataFrame 0 successfully saved!
```java
package org.reptile;

public class Reptile extends AnimalMarilho {
    protected int age;
    protected String name;

    @Override
    public int hashCode() {
        final int prime = 31;
        int result = 1;
        result = prime * result + ((action == null) ? 0 : action.hashCode());
        result = prime * result + age;
        result = prime * result + (name == null ? 0 : name.hashCode());
        result = prime * result + spead;
        return result;
    }

    public String getName() {
        return name;
    }

    public void setName(String name) {
        this.name = name;
    }
}
```
```java
	//Refactored code
	protected int age;
	protected int name;
	protected int spead;

```
DataFrame 1 successfully saved!
```java
package org.reptile;

public class Reptile extends AnimalMarilho 

In [50]:
prompt1 = construct_simple_prompt(join_file(data_frames[1]["beforeRefactoring.file"][0]))
print(prompt1)
LLM_answers = askChatGPT(prompt1)

Refactor the following class:
package org.animals;

public class Labrador extends Dog {
	
}
ChatCompletion(id='chatcmpl-8dYqcm6CmxsnLIKTG3cSlqve3tF0f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='package org.animals;\n\npublic class Labrador extends Dog {\n    // No need to alter the class content\n}', role='assistant', function_call=None, tool_calls=None))], created=1704439438, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_99cc374e39', usage=CompletionUsage(completion_tokens=22, prompt_tokens=55, total_tokens=77))


In [75]:
print(len(LLM_answers.choices[0].message.content))

101
